In [4]:
import argparse
import json
import cv2
import os
import numpy as np
import torch
import torchvision.transforms as TTR
from torch.utils.data import DataLoader, Dataset

In [28]:
 #从 taxonomy.json 文件中读取颜色映射
with open('./taxonomy.json', 'rb') as f:
    taxonomy_data = json.load(f)
    all_srgb_colormap = taxonomy_data['srgb_colormap']

# 根据 dms46 创建 srgb_colormap
dms46 = [
    1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 16, 17, 18, 19, 20, 21, 23,
    24, 26, 27, 29, 30, 32, 33, 34, 35, 36, 37, 38, 39, 41, 43, 44, 46, 47, 48, 49,
    50, 51, 52, 53, 56,
]
srgb_colormap = [
    all_srgb_colormap[i-1] for i in dms46
]
srgb_colormap.append([0, 0, 0])  # 添加黑色以表示无法检测的材料
srgb_colormap = np.array(srgb_colormap, dtype=np.uint8)
srgb_colormap

array([[  0,   0,   0],
       [188, 188, 137],
       [  0, 188,   0],
       [188, 188,   0],
       [  0,   0, 188],
       [188,   0, 188],
       [  0, 188, 188],
       [241, 241, 241],
       [  0, 137, 137],
       [225,   0,   0],
       [137, 188,   0],
       [225, 188,   0],
       [137,   0, 188],
       [  0, 225, 137],
       [225, 188, 188],
       [  0, 137,   0],
       [188, 137,   0],
       [137, 225, 188],
       [188, 137, 188],
       [  0, 137, 188],
       [  0, 225, 188],
       [188, 225, 188],
       [225, 137,   0],
       [137, 225,   0],
       [137, 137, 188],
       [225, 137, 188],
       [188,   0, 225],
       [  0,   0, 137],
       [188,   0,   0],
       [  0, 188, 137],
       [188,   0, 137],
       [  0,   0, 225],
       [225, 188, 137],
       [  0, 188, 225],
       [137,   0, 137],
       [137, 188, 137],
       [225, 225, 188],
       [225,   0, 225],
       [137, 188, 225],
       [225, 188, 225],
       [  0, 137, 225],
       [188, 137

In [27]:
print(prediction[0].shape)

(667, 1000)


In [33]:
# 设置预训练模型文件的路径
model_path = "./DMS46_v1.pt"

# 加载预训练模型
model = torch.jit.load(model_path)
model.eval()

# 输入图像路径
input_image_path = "./input_image.jpg"

# 读取输入图像
input_image = cv2.imread(input_image_path)
input_image = cv2.cvtColor(input_image, cv2.COLOR_BGR2RGB)

# 预处理图像
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
value_scale = 255
mean = [item * value_scale for item in mean]
std = [item * value_scale for item in std]

input_tensor = torch.from_numpy(input_image.transpose((2, 0, 1))).float()
input_tensor = TTR.Normalize(mean, std)(input_tensor)
input_tensor = input_tensor.unsqueeze(0)

# 进行预测
with torch.no_grad():
    prediction = model(input_tensor)[0].data.cpu().numpy()

# 应用颜色映射
# 将预测矩阵转换为二维数组
prediction = np.argmax(prediction, axis=0)

# 定义一个将类别索引转换为颜色的函数
def index_to_color(index):
    return srgb_colormap[index]

# 使用 np.vectorize 将函数应用到整个预测矩阵上
index_to_color_vec = np.vectorize(index_to_color, signature='()->(n)')
predicted_colored = index_to_color_vec(prediction)


# predicted_colored = np.take(srgb_colormap, prediction[0], axis=0)

# 保存预测结果
# output_image_path = "./predicted_image.png"
# cv2.imwrite(output_image_path, predicted_colored[..., ::-1])

# 设置缩放比例，例如 0.5 表示将宽度和高度缩小一半
scale_factor = 0.5
width = int(predicted_colored.shape[1] * scale_factor)
height = int(predicted_colored.shape[0] * scale_factor)
dim = (width, height)

# 将预测结果转换为 np.uint8 类型
predicted_colored_uint8 = predicted_colored.astype(np.uint8)

# 缩放图像
resized_predicted_colored = cv2.resize(predicted_colored_uint8, dim, interpolation=cv2.INTER_AREA)

# 重新定义预测结果数组的形状，使其与缩放后的输入图像的形状相匹配
resized_predicted_colored = resized_predicted_colored.reshape(height, width, 3)

# 保存预测结果
output_image_path = "./predicted_image.png"
cv2.imwrite(output_image_path, resized_predicted_colored[..., ::-1])





# 在这里添加将分析结果保存到 CSV 文件的代码


error: OpenCV(4.7.0) /Users/runner/work/opencv-python/opencv-python/opencv/modules/imgproc/src/resize.cpp:4065: error: (-215:Assertion failed) inv_scale_x > 0 in function 'resize'


In [26]:
print(np.max(prediction))


0
